### 1. Create a mirror function for every rule that returns 
- as of now it returns binary variables. later, weights can be assigned and scores can be averaged
- 1 = compliant, 0 = not-compliant

In [ ]:
import os
import spacy
import regex as re

from typing import List, Set, Generator, Dict, Any
from spacy.tokens import Doc, Token
from word2num_de import word_to_number
from sentence_transformers import SentenceTransformer, util
import language_tool_python
from sentence_transformers import SentenceTransformer, util, SimilarityFunction
from sentence_transformers.evaluation import SimilarityFunction
from sklearn.metrics.pairwise import cosine_similarity


from helper import SUBORDINATE_MARKERS, COORD_CONJ

W0914 15:56:26.447000 18892 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/kristina/anaconda3/envs/trl-env/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
nlp = spacy.load("de_core_news_lg")
from german_compound_splitter import comp_split

utf8_file = os.path.join("german_dict", "german_utf8.dic")
ahoc = comp_split.read_dictionary_from_file(utf8_file) #activate the compound_spliter

Loading data file - german_dict/german_utf8.dic


## Generate Checker functions for compound and converted numbers

In [ ]:
def ok_numbers_converted(doc: Doc) -> float:
    """
    A single, self-contained function to check for unconverted numbers.
    This has NO external dependencies to ensure the correct logic is always executed.
    """
    # All necessary constants are defined INSIDE this function
    NUMBER_DICT = {
    # Ordinals
    "erster": "1.", "zweiter": "2.", "dritter": "3.", "vierter": "4.", "fünfter": "5.", "sechster": "6.", "siebter": "7.",
    "achter": "8.", "neunter": "9.", "zehnter": "10.", "elfter": "11.", "zwölfter": "12.",
    # Fractions
    "halb": "0.5", "eineinhalb": "1.5", "zweieinhalb": "2.5", "dreieinhalb": "3.5", "viereinhalb": "4.5",
    "fünfeinhalb": "5.5", "sechseinhalb": "6.5", "siebeneinhalb": "7.5", "achteinhalb": "8.5", "neuneinhalb": "9.5", "zehneinhalb": "10.5",
}
    RE_NUMERIC = re.compile(r"^\d+([.,]\d+)?$")
    RE_ORDINAL = re.compile(r"^\d+\.$")

    # --- Internal helper to check each token

    def _is_unconverted_internal(token: Token) -> bool:
        """Internal helper to check a single token."""
        # This is the 'like_num' logic, using token attributes correctly
        text, lemma = token.text, token.lemma_
        text_lower, lemma_lower = text.lower(), lemma.lower()
        is_like_num = False
        if lemma_lower in NUMBER_DICT or text_lower in NUMBER_DICT: is_like_num = True
        elif RE_NUMERIC.match(text) or RE_ORDINAL.match(text): is_like_num = True
        elif text.isdigit(): is_like_num = True
        else:
            try:
                word_to_number(lemma_lower)
                is_like_num = True
            except Exception:
                try:
                    word_to_number(text_lower)
                    is_like_num = True
                except Exception:
                    is_like_num = False
        
        # 'is_number' logic
        is_a_number = False
        if token.text.lower() == "ein" and token.pos_ != "NUM":
            is_a_number = False
        else:
            is_a_number = is_like_num or token.pos_ == "NUM"

        # 'is_number_word_that_should_be_converted' logic
        if not is_a_number:
            return False
        return not token.text.isdigit()

    # --- Main calculation
    violating_tokens = [token for token in doc if _is_unconverted_internal(token)]
    violation_count = len(violating_tokens)
    total_tokens = len(doc)
    
    score = 1.0 # Default score is perfect (1.0)
    if total_tokens > 0: # Calculate penalty based on violations
        penalty = min(1.0, violation_count / total_tokens) # Normalize the penalty
        score = 1.0 - penalty

    return score # if I want to return all for tracking/debugging violation_count, total_tokens, violating_tokens

In [ ]:
def has_unsplit_compound(doc: spacy.tokens.Doc, ahoc: set) -> bool:
    """
    Checks if a document contains any unsplit compound nouns
    that should be simplified according to the given rules.

    This function iterates through each token in a spaCy Doc object and
    applies a set of heuristics to determine if it is a compound that
    should have been split but wasn't.

    Args:
        doc (spacy.tokens.Doc): The spaCy document object to check.
        ahoc (set): A lexicon or set of valid German words for checking
                    the validity of split parts.

    Returns:
        bool: True if at least one unsplit compound is found, False otherwise.
    """
    for token in doc:
        # Step 1: Preliminary checks on the token based on your logic.
        # This combines the logic from your `check_compound_split` and
        # `should_split` functions.
        if token.pos_ != "NOUN" or token.ent_type_ in {"PER", "LOC", "ORG"}:
            continue

        # Step 2: Attempt to split the compound using your splitter.
        parts =  comp_split.dissect(token.text, ahoc)
        
        # Step 3: Check if the token is a compound that can be split.
        if len(parts) < 2:
            continue
        
        # Step 4: Apply your "Konvens" rule check.
        # This rule suggests that if both the first and last parts of a
        # compound are short (<= 4 characters), it's not considered
        # "hard to read" and shouldn't be flagged as a violation.
        if len(parts[0]) <= 4 and len(parts[-1]) <= 4:
            continue

        # Step 5: Check if the split parts are valid words in the lexicon.
        # This ensures we don't try to split non-compounds or proper nouns
        # that aren't marked as named entities.
        valid_parts_count = sum(p.lower() in ahoc for p in parts)
        
        # Step 6: If a majority of the parts are valid, it's a compound that
        # should have been split. We've found a violation.
        if valid_parts_count / len(parts) >= 0.8:
            # Processing Optimization Changes
            # print(f"Violation detected: Found unsplit compound '{token.text}'")
            return True
            
    # If the loop completes without finding any violations, the rule is followed.
    return False

In [5]:
def has_apposition(doc: spacy.tokens.Doc) -> bool: #regex finds likely comma apposition
    if any(tok.dep_ == "app" for tok in doc):
        return True
    # Fallback: regex check for ', ... ,'
    # Only trigger if pattern matches (not followed by "die", "der", etc.)
    match = re.search(r', (?!die |der |das |und |aber |weil |obwohl )[^,]+,', doc.text)
    return bool(match)

def has_subordinate_clause(doc: spacy.tokens.Doc) -> bool:
    for tok in doc:
        if (tok.text.lower() in SUBORDINATE_MARKERS and 
            (tok.dep_ in "cp" or tok.pos_ == "SCONJ")):
            return True
    return False

def has_coordinate_clause(doc: spacy.tokens.Doc) -> bool:
    """Check if the document contains a coordinate clause."""
    return any(tok.dep_ == "cd" and tok.text.lower() in COORD_CONJ for tok in doc)

def has_disallowed_tense(doc: spacy.tokens.Doc) -> bool:
    for tok in doc:
        if tok.pos_ in ("VERB", "AUX"):
            tense = tok.morph.get("Tense")
            form = tok.morph.get("VerbForm")
            mood = tok.morph.get("Mood")
            if ("Pres" not in tense and "Part" not in form) or ("Sub" in mood):
                return True
    return False

def is_passive(doc: spacy.tokens.Doc) -> bool:
    # Werden + participle: Vorgangspassiv (event passive)
    has_werden = any(tok.lemma_ == "werden" and tok.pos_ == "AUX" for tok in doc)
    has_participle = any(tok.pos_ == "VERB" and "Part" in tok.morph.get("VerbForm", []) for tok in doc)
    if has_werden and has_participle:
        return True
    # Sein + participle: Zustandspassiv (state passive), only for transitive verbs!
    has_sein = any(tok.lemma_ == "sein" and tok.pos_ == "AUX" for tok in doc)
    if has_sein and has_participle:
        # Check: is the main verb transitive (does it take an object)?
        if any(tok.dep_ in {"oa", "obj"} for tok in doc):  # object present
            return True
    return False

# Collect the main functions to compute reward function

### Example Sentence

- has_apposition(doc) returns True/False
- float(has_apposition(doc)) returns 1.0 (True, violated), 0.0 (False, compliant)
- what does the ok counterpart do?
  - flips the perspective

| original text flag | has_xy -> float | ok_no_xy | meaning |
| --- | --- | --- | --- |
| apposition present | 1.0 | 0.0 | not okay |
| no apposition | 0.0 | 1.0 | okay |

| Function | Returns | Meaning |
| --- | --- | --- |
| float(has_xy(...)) | 1.0 if violation, 0.0 if compliant | "Badness Score"|
| ok_no_xy | 1.0 if compliant, 0.0 if violation | "Goodness Score" |

In [6]:
def ok_no_apposition(doc)           -> float: return 1.0 - float(has_apposition(doc))
def ok_no_subordinate_clause(doc)   -> float: return 1.0 - float(has_subordinate_clause(doc))
#def ok_no_coordinate_clause(doc)    -> float: return 1.0 - float(has_coordinate_clause(doc))
def ok_active_voice(doc)            -> float: return 1.0 - float(is_passive(doc))
def ok_allowed_verb_tense(doc)      -> float: return 1.0 - float(has_disallowed_tense(doc))

In [ ]:
# 1. Create a single "source of truth" generator function
def _find_unsplit_compounds_gen(doc: spacy.tokens.Doc, ahoc: Set[str]) -> Generator[spacy.tokens.Token, None, None]:
    """
    A generator that yields each token that is an unsplit compound violation.
    This contains the core, shared logic.
    """
    for token in doc:
        # Initial checks: must be a NOUN and not a named entity
        if token.pos_ != "NOUN" or token.ent_type_ in {"PER", "LOC", "ORG"}:
            continue
        if not token.text or not token.text.strip() or not any(char.isalpha() for char in token.text): #adding fix
            continue

        
        parts = [] # Default to an empty list
        try:
            parts = comp_split.dissect(token.text, ahoc)
        except IndexError:
            continue
        
        if not parts or len(parts) < 2:
            continue
        if len(parts[0]) <= 4 and len(parts[-1]) <= 4:
            continue

        valid_parts_count = sum(p.lower() in ahoc for p in parts)
        
        if valid_parts_count / len(parts) >= 0.8:
            yield token # Yield the violating token and continue the loop

def count_unsplit_compounds(doc: spacy.tokens.Doc, ahoc: Set[str]) -> int:
    """Counts ALL unsplit compounds"""
    return sum(1 for _ in _find_unsplit_compounds_gen(doc, ahoc))

def find_all_unsplit_compounds(doc: spacy.tokens.Doc, ahoc: Set[str]) -> list[spacy.tokens.Token]:
    """Gets a list of all violating tokens. Useful for debugging."""
    return list(_find_unsplit_compounds_gen(doc, ahoc))

def ok_no_compounds(doc: spacy.tokens.Doc) -> float:
    """
    Computes a compliance score (0-1) for compound splitting.
    """
    violation_count = count_unsplit_compounds(doc, ahoc)
    noun_count = len([token for token in doc if token.pos_ == "NOUN"])
    
    if noun_count == 0:
        return 1.0
    
    penalty = min(1.0, violation_count / noun_count)
    return 1.0 - penalty

### 2. Collect them all in a single rule wrapper

In [ ]:
RULE_CHECKS = {
    "apposition"        : ok_no_apposition,
    "subord_clause"     : ok_no_subordinate_clause,
    #"coord_clause"      : ok_no_coordinate_clause,
    "voice_active"      : ok_active_voice,
    "verb_tense"        : ok_allowed_verb_tense,
    "no_compounds"      : ok_no_compounds,
    "numbers_converted" : ok_numbers_converted,
}
RULE_WEIGHTS = { # must sum to 1.0
   "apposition"        : 0.20,   
   "subord_clause"     : 0.10,   
   "voice_active"      : 0.15,   
   "verb_tense"        : 0.25,   
   "no_compounds"      : 0.20,   
   "numbers_converted" : 0.10,   
}

In [ ]:
# Normalize weights to ensure they sum to 1.0
total_weight = sum(RULE_WEIGHTS.values())
if total_weight != 1.0:
    for key in RULE_WEIGHTS:
        RULE_WEIGHTS[key] /= total_weight
    print("Warning: RULE_WEIGHTS did not sum to 1.0. They have been normalized.")
print(total_weight)

### 3. Create the scorer

In [ ]:
def rule_compliance_score(text: str) -> float:
    """
    Computes a weighted 0-1 compliance score for a simplified sentence.
    This is the core function for rule-based rewards.
    """
    doc = nlp(text)
    total_score = 0.0
    
    # Iterate through each rule and its corresponding checker function
    for name, check_func in RULE_CHECKS.items():
        # Call the checker function and get the score
        score = check_func(doc)
        # Clamp score for required range
        score = max(0.0, min(1.0, score))
        
        # Add the weighted score to the total
        total_score += RULE_WEIGHTS[name] * score
        
    return total_score

### 4. Compute a meaning-preservation score
- embedding-level similarity which works cross-lingually without refences
- https://sbert.net/docs/sentence_transformer/usage/semantic_textual_similarity.html

In [ ]:
print("Loading BERT model...")
model = SentenceTransformer('deepset/gbert-large') #-large -small
model.similarity_fn_name = SimilarityFunction.DOT_PRODUCT #dot product is faster and more efficient
print(f"Model similarity function set to: '{model.similarity_fn_name}'")
print("Model loaded.")

Loading SBERT model...


No sentence-transformers model found with name deepset/gbert-base. Creating a new one with mean pooling.
/Users/kristina/anaconda3/envs/trl-env/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Model similarity function set to: 'dot'
Model loaded.


In [ ]:
def calculate_semantic_similarity(original_sentence: str, simplified_sentence: str) -> float:
    """
    Calculates the meaning preservation score using SBERT embeddings.
    Returns a score between 0.0 and 1.0.
    """
    # Encode sentences with normalization for faster comparison
    emb_original = model.encode(original_sentence, normalize_embeddings=True)
    emb_simplified = model.encode(simplified_sentence, normalize_embeddings=True)
    
    
    # 2. Calculate the cosine similarity between the two vectors. This returns a tensor.
    similarity_score = model.similarity(emb_original, emb_simplified).item()
    
    #map the [-1,1] range to [0,1]
    return (similarity_score + 1.0) / 2.0

# Compute Grammar Score

In [ ]:
tool = language_tool_python.LanguageTool('de')

In [ ]:
def calculate_grammar_score(text: str) -> float:
    """
    Computes a grammatical compliance score (0-1) for a sentence.
    A score of 1.0 means no errors were found.
    """
    error_count = 0
    
    if not text.strip():
        return 1.0 # Return perfect score for empty strings
    
    # Check the text for errors
    matches = tool.check(text)

    grammar_errors = [match for match in matches if match.category == 'GRAMMAR']


    if matches:
        #print(f"Found {len(grammar_errors)} grammar errors")
        error_count = len(grammar_errors) #track number of errors
        #for match in matches:
        #    print(f"Error: {match.message}")
    
    # Get a base for normalization, by counting number of tokens
    token_count = len(text.split())
    
    if token_count == 0:
        return 1.0
        
    # Calculate the penalty. I use a simple normalization. Capped at 1. Make sure to be less harsh on short sentences
    #penalty = min(1.0, error_count / token_count) ## linear penalty
    penalty = error_count / (error_count + token_count) #softer penalty, as errors increase, penalty -> 1.0 asymptotically
    
    # Return the compliance score
    return 1.0 - penalty

### 4. Plug into the global reward function

Assessing four different weight distribution options:

Baseline (0.5, 0.3, 0.2)

Rules-heavy (0.6, 0.2, 0.2)

Meaning-heavy (0.3, 0.6, 0.1)

Grammar-sensitive (0.4, 0.2, 0.4)

In [ ]:
# weights = [
#     {"rules_score": 0.5, "meaning_score": 0.3, "grammar_score": 0.2},
#     {"rules_score": 0.6, "meaning_score": 0.2, "grammar_score": 0.2},
#     {"rules_score": 0.3, "meaning_score": 0.6, "grammar_score": 0.1},
#     {"rules_score": 0.4, "meaning_score": 0.2, "grammar_score": 0.4},
# ]

# weights = {
#     "rules_score":   0.5,
#     "meaning_score": 0.3,
#     "grammar_score": 0.2,
# }


In [ ]:
def compute_reward(src: str, simpl: str, weights: dict) -> float:
    """
    Combines the rule compliance, meaning preservation and grammar scores into one.
    
    Args:
        src (str): The original, complex sentence.
        simpl (str): The simplified output sentence.
    
    Returns:
        float: The final reward score.
    """

    # Ensure weights sum is very close avoiding floating point issues
    if abs(sum(weights.values()) - 1.0) > 1e-9:
        
        raise ValueError(f"Reward weights must sum to 1.0, but they sum to {sum(weights.values())}")
    
    
    r_rules   = rule_compliance_score(simpl)
    r_meaning = calculate_semantic_similarity(src, simpl)
    r_grammar = calculate_grammar_score(simpl)

    reward = (weights["rules_score"]   * r_rules +
              weights["meaning_score"] * r_meaning +
              weights["grammar_score"] * r_grammar)
    
    # Clamp final combined reward
    return float(max(0.0, min(1.0, reward)))